In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import os
import warnings

# 파일 불러와서 열이름 바꾸기
korean, long_term_frgn, short_term_frgn, resident, card_cnt, card_amt, waste_cnt
(이거 예시는 card_amt임)

In [2]:
df = pd.read_csv("files\\data_preprocessing\\long_term_frgn_preprocessing.csv", encoding='cp949')

# 시계열 피처단위로 변경
df['base_date'] = pd.to_datetime(df['base_date'])

# base_date를 index로 설정
df.index = df['base_date']
df.columns = ['base_date', 'emd_cd', 'long_term_frgn']  
df

,base_date,emd_cd,long_term_frgn
base_date,,,
2018-01-31,2018-01-31,50110250,427986.3363
2018-02-28,2018-02-28,50110250,427240.8119
2018-03-31,2018-03-31,50110250,513638.2631
2018-04-30,2018-04-30,50110250,474601.3510
2018-05-31,2018-05-31,50110250,395262.5483
...,...,...,...
2021-02-28,2021-02-28,50130620,109325.4866
2021-03-31,2021-03-31,50130620,118258.8891
2021-04-30,2021-04-30,50130620,129227.7480


In [3]:
# 필요 없는 열 제거
data = df.drop(['base_date'], 1)    
data

,emd_cd,long_term_frgn
base_date,,
2018-01-31,50110250,427986.3363
2018-02-28,50110250,427240.8119
2018-03-31,50110250,513638.2631
2018-04-30,50110250,474601.3510
2018-05-31,50110250,395262.5483
...,...,...
2021-02-28,50130620,109325.4866
2021-03-31,50130620,118258.8891
2021-04-30,50130620,129227.7480


In [4]:
# 한 행정동의 행만 추출
sub_area = data['emd_cd'] == 50130259

# base_date, card_amt 만 갖는 df 생성
sub_df = data[sub_area]
sub_df = sub_df.drop('emd_cd', 1)
sub_df

,long_term_frgn
base_date,
2018-01-31,201779.6551
2018-02-28,182328.5481
2018-03-31,204358.8023
2018-04-30,221148.6409
2018-05-31,183450.3552
2018-06-30,223060.6966
2018-07-31,278243.5444
2018-08-31,286227.0196
2018-09-30,256437.9674


In [5]:
# series 형으로 변환
sub_df.to_dict('series')

{'long_term_frgn': base_date
 2018-01-31    201779.6551
 2018-02-28    182328.5481
 2018-03-31    204358.8023
 2018-04-30    221148.6409
 2018-05-31    183450.3552
 2018-06-30    223060.6966
 2018-07-31    278243.5444
 2018-08-31    286227.0196
 2018-09-30    256437.9674
 2018-10-31    206134.5213
 2018-11-30    177161.5667
 2018-12-31    214030.3816
 2019-01-31    228862.1419
 2019-02-28    214574.8246
 2019-03-31    307503.8846
 2019-04-30    206292.3897
 2019-05-31    240216.3143
 2019-06-30    244797.3621
 2019-07-31    318763.8508
 2019-08-31    434533.5644
 2019-09-30    247170.5629
 2019-10-31    226539.7222
 2019-11-30    206893.5308
 2019-12-31    211849.4175
 2020-01-31    263284.9470
 2020-02-29    178845.5857
 2020-03-31    282394.7938
 2020-04-30    204955.8372
 2020-05-31    197820.8133
 2020-06-30    194740.8436
 2020-07-31    339717.0794
 2020-08-31    359710.7479
 2020-09-30    212122.2631
 2020-10-31    193854.5209
 2020-11-30    216940.9166
 2020-12-31    231428.3790

# ARIMA

In [6]:
# ARIMA에 적합한 float로 바꾸는 과정
sub_df_float = sub_df[:].astype(np.float)

In [7]:
# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
from statsmodels.tsa.stattools import adfuller

result = adfuller(sub_df)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))

ADF Statistic : -2.038746
p-value : 0.269847
Critical Values :
	1%: -3.661429
	5%: -2.960525
	10%: -2.619319


In [8]:
# 차분
diff_1 = sub_df_float.diff(periods=1).iloc[1:]

# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
result = adfuller(diff_1)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))

ADF Statistic : -4.732461
p-value : 0.000073
Critical Values :
	1%: -3.669920
	5%: -2.964071
	10%: -2.621171


In [9]:
from pmdarima.arima import auto_arima

model_arima= auto_arima(sub_df,trace=True, error_action='ignore',suppress_warnings=True,stepwise=False,seasonal=True)

model_arima.fit(sub_df)

 ARIMA(0,0,0)(0,0,0)[1] intercept   : AIC=1035.670, Time=0.01 sec
 ARIMA(0,0,1)(0,0,0)[1] intercept   : AIC=1034.221, Time=0.10 sec
 ARIMA(0,0,2)(0,0,0)[1] intercept   : AIC=1036.623, Time=0.04 sec
 ARIMA(0,0,3)(0,0,0)[1] intercept   : AIC=1034.965, Time=0.10 sec
 ARIMA(0,0,4)(0,0,0)[1] intercept   : AIC=1032.065, Time=0.13 sec
 ARIMA(0,0,5)(0,0,0)[1] intercept   : AIC=1030.898, Time=0.15 sec
 ARIMA(1,0,0)(0,0,0)[1] intercept   : AIC=1034.791, Time=0.03 sec
 ARIMA(1,0,1)(0,0,0)[1] intercept   : AIC=1036.192, Time=0.05 sec
 ARIMA(1,0,2)(0,0,0)[1] intercept   : AIC=1038.422, Time=0.07 sec
 ARIMA(1,0,3)(0,0,0)[1] intercept   : AIC=1038.166, Time=0.12 sec
 ARIMA(1,0,4)(0,0,0)[1] intercept   : AIC=1034.654, Time=0.13 sec
 ARIMA(2,0,0)(0,0,0)[1] intercept   : AIC=1035.470, Time=0.05 sec
 ARIMA(2,0,1)(0,0,0)[1] intercept   : AIC=1038.072, Time=0.18 sec
 ARIMA(2,0,2)(0,0,0)[1] intercept   : AIC=1037.726, Time=0.24 sec
 ARIMA(2,0,3)(0,0,0)[1] intercept   : AIC=1036.207, Time=0.21 sec
 ARIMA(3,0

ARIMA(order=(0, 0, 5), scoring_args={}, seasonal_order=(0, 0, 0, 1),
      suppress_warnings=True)

In [10]:
# ARIMA 모델 
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm

# (AR=2, 차분=1, MA=2) 파라미터로 ARIMA 모델을 학습합니다.
model = ARIMA(sub_df.long_term_frgn.values, order=(0,0,5))

#trend : constant를 가지고 있는지, c - constant / nc - no constant
#disp : 수렴 정보를 나타냄
model_fit = model.fit(trend='c', full_output=True, disp=True)
print(model_fit.summary())

C:\Users\soeun\anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)


                              ARMA Model Results                              
Dep. Variable:                      y   No. Observations:                   42
Model:                     ARMA(0, 5)   Log Likelihood                -508.309
Method:                       css-mle   S.D. of innovations          41037.346
Date:                Sat, 11 Sep 2021   AIC                           1030.618
Time:                        04:33:15   BIC                           1042.782
Sample:                             0   HQIC                          1035.077
                                                                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const       2.378e+05   7512.225     31.654      0.000    2.23e+05    2.53e+05
ma.L1.y        0.4358      0.168      2.598      0.009       0.107       0.765
ma.L2.y        0.1888      0.260      0.727      0.4

# 예측하기

In [11]:
# 2단위 이후의 예측결과
fore = model_fit.forecast(steps=2)
print(fore)

(array([238917.5235511 , 237789.21092112]), array([41037.34643032, 44765.74541953]), array([[158485.80252658, 319349.24457561],
       [150049.96215774, 325528.45968449]]))
